In [1]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math


np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def sample_first_prows(data, perc):
    return data.head(int(len(data)*(perc)))

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
# convert series to supervised learning
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [6]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/HomeC.csv', sep=',')
data = df.drop(labels=['icon','summary','cloudCover', 'House overall [kW]', 'time'], axis=1)
#data = data.loc[0: : 5]
data.dropna(inplace=True)
data = clean_dataset(data)
data.head()

/home/hugo/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,use [kW],gen [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
0,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
1,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
2,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
3,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
4,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


In [7]:
# frame as supervised learning
reframed = series_to_supervised_mimo(data, 1, 1)
reframed.head()

,use [kW](t-1),gen [kW](t-1),Dishwasher [kW](t-1),Furnace 1 [kW](t-1),Furnace 2 [kW](t-1),Home office [kW](t-1),Fridge [kW](t-1),Wine cellar [kW](t-1),Garage door [kW](t-1),Kitchen 12 [kW](t-1),...,temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
1,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
2,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
3,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
4,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0
5,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,...,36.14,0.62,10.0,29.26,1016.91,9.18,282.0,0.0,24.4,0.0


In [8]:
cols = reframed.columns
cols

Index(['use [kW](t-1)', 'gen [kW](t-1)', 'Dishwasher [kW](t-1)',
       'Furnace 1 [kW](t-1)', 'Furnace 2 [kW](t-1)', 'Home office [kW](t-1)',
       'Fridge [kW](t-1)', 'Wine cellar [kW](t-1)', 'Garage door [kW](t-1)',
       'Kitchen 12 [kW](t-1)', 'Kitchen 14 [kW](t-1)', 'Kitchen 38 [kW](t-1)',
       'Barn [kW](t-1)', 'Well [kW](t-1)', 'Microwave [kW](t-1)',
       'Living room [kW](t-1)', 'Solar [kW](t-1)', 'temperature(t-1)',
       'humidity(t-1)', 'visibility(t-1)', 'apparentTemperature(t-1)',
       'pressure(t-1)', 'windSpeed(t-1)', 'windBearing(t-1)',
       'precipIntensity(t-1)', 'dewPoint(t-1)', 'precipProbability(t-1)',
       'use [kW](t)', 'gen [kW](t)', 'Dishwasher [kW](t)', 'Furnace 1 [kW](t)',
       'Furnace 2 [kW](t)', 'Home office [kW](t)', 'Fridge [kW](t)',
       'Wine cellar [kW](t)', 'Garage door [kW](t)', 'Kitchen 12 [kW](t)',
       'Kitchen 14 [kW](t)', 'Kitchen 38 [kW](t)', 'Barn [kW](t)',
       'Well [kW](t)', 'Microwave [kW](t)', 'Living room [kW](t)',

In [9]:
df_forecats_columns = ['use [kW](t)', 'gen [kW](t)', 'Dishwasher [kW](t)', 'Furnace 1 [kW](t)',
       'Furnace 2 [kW](t)', 'Home office [kW](t)', 'Fridge [kW](t)',
       'Wine cellar [kW](t)', 'Garage door [kW](t)', 'Kitchen 12 [kW](t)',
       'Kitchen 14 [kW](t)', 'Kitchen 38 [kW](t)', 'Barn [kW](t)',
       'Well [kW](t)', 'Microwave [kW](t)', 'Living room [kW](t)',
       'Solar [kW](t)', 'temperature(t)', 'humidity(t)', 'visibility(t)',
       'apparentTemperature(t)', 'pressure(t)', 'windSpeed(t)',
       'windBearing(t)', 'precipIntensity(t)', 'dewPoint(t)',
       'precipProbability(t)']

In [10]:
def sliding_window_pca(data,n_windows,train_size,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
            df_train = ttrain.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            df_test = ttest.loc[:,'use [kW](t-1)':'precipProbability(t-1)']
            df_original = ttest.loc[:,'use [kW](t)':'precipProbability(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(df_train,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,df_test,steps_ahead,transformation)
            
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
            for col in columns:  
                original = df_original[col].values
                forecast = df_forecast[col].values
#                 original = original[:len(original)-1]
#                 forecast = forecast[1:]
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [11]:
num_components_pca = 3
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [12]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 16796
--------------------
training window 33592
--------------------
training window 50388
--------------------
training window 67184
--------------------
training window 83980
--------------------
training window 100776
--------------------
training window 117572
--------------------
training window 134368
--------------------
training window 151164
--------------------
training window 167960
--------------------
training window 184756
--------------------
training window 201552
--------------------
training window 218348
--------------------
training window 235144
--------------------
training window 251940
--------------------
training window 268736
--------------------
training window 285532
--------------------
training window 302328
--------------------
training window 319124
--------------------
training window 335920
--------------------
training window 352716
--------------------
training window 36951

In [13]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

kpca_result.to_csv (r'all_windows_kpca_mo_ensfts_kaggle_sh.csv', index = False, header=True)

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [14]:
print("Statistics MIMO KPCA-NSFTS(test): ")
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,use [kW](t),0.377,0.188,89.886,0.693,13.547,0.056,0.219,0.139,87.436,0.236,6.403,0.016
1,gen [kW](t),0.034,0.023,inf,0.914,47.233,0.071,0.014,0.012,NaN,0.037,10.007,0.026
2,Dishwasher [kW](t),0.053,0.011,inf,-2.511,93.420,0.039,0.015,0.005,NaN,18.748,2.761,0.011
3,Furnace 1 [kW](t),0.060,0.026,50.492,0.776,18.685,0.074,0.030,0.021,37.450,0.134,14.274,0.047
4,Furnace 2 [kW](t),0.064,0.032,26.424,0.801,11.448,0.106,0.048,0.030,22.608,0.069,10.321,0.061
5,Home office [kW](t),0.021,0.012,19.760,0.902,10.255,0.038,0.010,0.008,12.612,0.213,6.818,0.019
6,Fridge [kW](t),0.023,0.008,66.946,0.907,20.974,0.040,0.003,0.003,25.484,0.017,7.015,0.013
7,Wine cellar [kW](t),0.010,0.005,40.424,0.959,16.240,0.054,0.006,0.003,25.075,0.025,8.656,0.025
8,Garage door [kW](t),0.011,0.001,5.079,0.119,1.951,0.036,0.009,0.001,6.861,0.320,1.753,0.011
9,Kitchen 12 [kW](t),0.005,0.001,inf,0.451,14.969,0.014,0.009,0.003,NaN,0.572,23.278,0.019


In [16]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_kaggle_sh_with_std.csv', index = False, header=True)

## Sliding Window: MIMO PCA-NSFTS

In [17]:
num_components_pca = 8 #12 #8
order_fts_model = 1
npart = 60
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order_fts_model, npart,
                                            gamma, fts_model, memory_window_error)

In [18]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 16796
--------------------
training window 33592
--------------------
training window 50388
--------------------
training window 67184
--------------------
training window 83980
--------------------
training window 100776
--------------------
training window 117572
--------------------
training window 134368


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 151164
--------------------
training window 167960


/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:83: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100
/home/hugo/anaconda3/lib/python3.8/site-packages/pyFTS/benchmarks/Measures.py:102: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(forecasts - targets) / (np.abs(forecasts) + abs(targets))) * 100


--------------------
training window 184756
--------------------
training window 201552
--------------------
training window 218348
--------------------
training window 235144
--------------------
training window 251940
--------------------
training window 268736
--------------------
training window 285532
--------------------
training window 302328
--------------------
training window 319124
--------------------
training window 335920
--------------------
training window 352716
--------------------
training window 369512
--------------------
training window 386308
--------------------
training window 403104
--------------------
training window 419900
--------------------
training window 436696
--------------------
training window 453492
--------------------
training window 470288
--------------------
training window 487084


In [21]:
columns = list(df_forecats_columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

pca_result.to_csv (r'all_windows_pca_mo_ensfts_kaggle_sh.csv', index = False, header=True)

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,use [kW](t),0.538,0.389,211.078,0.434,25.496,0.080,0.331,0.262,239.860,0.292,10.965,0.028
1,gen [kW](t),0.057,0.045,inf,0.735,59.454,0.119,0.016,0.013,NaN,0.158,5.520,0.026
2,Dishwasher [kW](t),0.160,0.104,inf,0.154,96.315,0.117,0.048,0.042,NaN,0.294,3.111,0.035
3,Furnace 1 [kW](t),0.119,0.086,225.507,0.089,48.965,0.151,0.056,0.050,125.055,0.349,15.674,0.084
4,Furnace 2 [kW](t),0.111,0.083,81.318,0.272,28.354,0.190,0.078,0.064,58.588,0.226,17.689,0.091
5,Home office [kW](t),0.079,0.057,92.326,0.331,34.856,0.151,0.032,0.026,40.503,0.248,13.755,0.057
6,Fridge [kW](t),0.065,0.048,512.271,0.230,53.499,0.114,0.014,0.013,221.826,0.283,7.663,0.044
7,Wine cellar [kW](t),0.045,0.034,231.453,0.191,51.479,0.252,0.019,0.014,120.557,0.230,12.730,0.089
8,Garage door [kW](t),0.014,0.005,37.552,-0.079,13.683,0.041,0.013,0.006,46.586,0.281,9.650,0.013
9,Kitchen 12 [kW](t),0.007,0.004,inf,0.214,36.632,0.026,0.013,0.008,NaN,0.355,35.147,0.047


In [22]:
final_measures_pca.to_csv(r'pca_mo_ensfts_kaggle_sh_with_std.csv', index = False, header=True)

In [24]:
#final_measures_pca.to_csv (r'pca_mo_ensfts_kaggle_sh_2.csv', index = False, header=True)